# Tag the Tweet!

Tagging a Tweet based on its context! Using Recurrent Neural Networks, NGrams.

# Problem Statement

        Tag the Tweet is an Application built on python, which understands all the tweets of the trending hashtags
        and predicts a hashtag for the given tweets.

In [2]:
import numpy as np
import twitter
import sys
import re, datetime, pandas as pd
import tweepy as tw
#from ResultMiner import TweetMiner

# Step - 01

We need a lot of data regarding the Tweets! Hence I am using `tweepy` to authenticate my Twitter Keys and use the API to retrieve the required data.

In [3]:
MAX_HASHTAGS_COUNT = 3
MIN_HASHTAGS_COUNT = 2
WOE_ID = 23424977
TWEET_LIMIT = 125

twitter_details = {
    'consumer_key'        : "pNPFulq7iU1AwsH8WKXcM3VhZ",
    'consumer_secret'     : "k6weUOaZxYSbBBLCBB5DsshL8iUDWIOLY5WlHK555kHRWOinuQ",
    'access_token_key'    : "782632596-Zx3pED7JikyG7dBx1a6oIqyQh60qeSMUXHAqjfhg",
    'access_token_secret' : "DoqjnFjyJIqEEsqUVtnyWX2KYyswA9rTCPtJr9GGfyTHL"
}

twitter_api = twitter.Api(consumer_key= twitter_details['consumer_key'],
                  consumer_secret     = twitter_details['consumer_secret'],
                  access_token_key    = twitter_details['access_token_key'],
                  access_token_secret = twitter_details['access_token_secret'])

auth = tw.OAuthHandler(twitter_details['consumer_key'], twitter_details['consumer_secret'])
auth.set_access_token(twitter_details['access_token_key'], twitter_details['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
class TweetMiner():

    def __init__(self, api, tweet_limit):
        
        self.api = api
        self.tweet_limit = tweet_limit
        
        
    def retrieve_data(self, tag, WOE_ID, page_limit):
        
        cleaned_data = []
        latest_max_id = False
        for i in range(page_limit):
            tweets = tw.Cursor(self.api.search,
                     q=tag,
                     lang="en",
                     max_id = latest_max_id,
                     since=2018-1-1).items(self.tweet_limit)
            
            for tweet in tweets:
                try:
                    data = {
                        'id' : tweet.id,
                        'user_name' : tweet.user.screen_name,
                        'tweet_description' : tweet.text,
                        'retweet_count' : tweet.retweet_count,
                        'created_at' : tweet.created_at,
                        'hashtag' : tag
                    }
                except:
                    data = {
                        'id' : tweet.id,
                        'user_name' : tweet.user.screen_name,
                        'tweet_description' : tweet.text,
                        'retweet_count' : 0,
                        'created_at' : tweet.created_at,
                        'hashtag' : tag
                    }
                latest_max_id = tweet.id
                cleaned_data.append(data)
        
        return cleaned_data

In [5]:
tweet_miner = TweetMiner(api, TWEET_LIMIT)

In [6]:
def get_trending_hashtags(trends):
    trends = trends['trends']
    trending_hashtags = []
    for trend in trends:
        if trend['name'][0] == '#':
            trending_hashtags.append(trend)
    return trending_hashtags[:MAX_HASHTAGS_COUNT]

In [7]:
hashtags = get_trending_hashtags(api.trends_place(WOE_ID)[0])
data = []
if len(hashtags) < MAX_HASHTAGS_COUNT:
    print("Not enough hashtags to proceed")
    sys.exit()

# Tweet Retrievel from Trending HashTags
  We are trying to understand three popular hashtags by mining 1500 tweets each..
  Below are the pandas dataframes for each of these. Which we later try to merge back
  as on dataset X

In [8]:
print(hashtags[0])

{'name': '#TransDayOfVisibility', 'url': 'http://twitter.com/search?q=%23TransDayOfVisibility', 'promoted_content': None, 'query': '%23TransDayOfVisibility', 'tweet_volume': 419043}


In [9]:
data.append(tweet_miner.retrieve_data(hashtags[0]['name'], WOE_ID, 20))
pd.DataFrame(data[0])

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1377320513542967296,tobesoperfect43,#TransDayOfVisibility idc what people say you...,0,2021-03-31 18:02:54,#TransDayOfVisibility
1,1377320511991123972,pentalisk,#TransDayOfVisibility #transcreators\n\nhi im ...,0,2021-03-31 18:02:54,#TransDayOfVisibility
2,1377320511961755650,stoptiime,RT @honkkarl: #TransDayOfVisibility \nStole th...,7253,2021-03-31 18:02:54,#TransDayOfVisibility
3,1377320511160614915,okrasocialist,happy #TransDayOfVisibility \n\ndonate to whom...,0,2021-03-31 18:02:53,#TransDayOfVisibility
4,1377320511139676162,Rkh1209Randy,RT @ProgressTX: 💙 Trans Texans deserve love.\n...,23,2021-03-31 18:02:53,#TransDayOfVisibility
...,...,...,...,...,...,...
2495,1377320078388121607,eveningchicory,RT @HydeNikolaev: Happy #TransDayOfVisibility ...,545,2021-03-31 18:01:10,#TransDayOfVisibility
2496,1377320078199431174,fall_groove,RT @eliciadonze: I tried this overnight &amp; ...,219,2021-03-31 18:01:10,#TransDayOfVisibility
2497,1377320078174216202,julio_sch27,RT @gottmik: #TransDayOfVisibility MOOD https:...,695,2021-03-31 18:01:10,#TransDayOfVisibility
2498,1377320077767319553,fufupox,RT @maikolgaming: hii its #TransDayOfVisibilit...,2,2021-03-31 18:01:10,#TransDayOfVisibility


In [12]:
data.append(tweet_miner.retrieve_data(hashtags[1]['name'], WOE_ID, 20))
pd.DataFrame(data[1])

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1377334750025641986,thecoachkessler,RT @aliciatcrosby: Wishing a blessed &amp; bea...,3,2021-03-31 18:59:28,#TDOV
1,1377334748763262983,FadeKartoffel,"RT @sethpuertoluna: Hello #TDOV , I'm a mixed ...",717,2021-03-31 18:59:28,#TDOV
2,1377334747358109697,Lostikins,RT @Nicowo_: its #TransDayOfVisibility so im h...,13,2021-03-31 18:59:28,#TDOV
3,1377334745919483904,brookzyr,"gender is fake and so is police ""reform"" happy...",0,2021-03-31 18:59:27,#TDOV
4,1377334745781174274,earthto_emilyo,RT @lannwastaken: since it’s #TransDayOfVisibi...,1318,2021-03-31 18:59:27,#TDOV
...,...,...,...,...,...,...
2495,1377331087974752259,Racingkats16,RT @moobIoom: happy #TransDayOfVisibility my n...,94,2021-03-31 18:44:55,#TDOV
2496,1377331083369271298,witchchick128,RT @vhsghostricks: #TransDayOfVisibility #TDOV...,27,2021-03-31 18:44:54,#TDOV
2497,1377331081205063682,monet5280,RT @joshrogin: Trans rights are human rights. ...,12,2021-03-31 18:44:53,#TDOV
2498,1377331080655740929,cervidrawn,RT @prismaticram: marie wishes you all a very ...,1,2021-03-31 18:44:53,#TDOV


In [13]:
data.append(tweet_miner.retrieve_data(hashtags[2]['name'], WOE_ID, 20))
pd.DataFrame(data[2])

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1377338541525590026,StkOptsInvestor,@MileyCyrus\n Money bag#15YearsofMiley #INVEST...,0,2021-03-31 19:14:32,#INVESTINYOU
1,1377338532340015104,vashappenxbiebs,RT @MileyCyrus: Nothing is more important than...,4038,2021-03-31 19:14:30,#INVESTINYOU
2,1377338528485548032,strafe_sells,@MileyCyrus @CashApp @CashApp @MileyCyrus $str...,0,2021-03-31 19:14:29,#INVESTINYOU
3,1377338526501457920,katxed,@MileyCyrus @CashApp $katxed Broadcom #invest...,0,2021-03-31 19:14:29,#INVESTINYOU
4,1377338517727027202,smilexfolklore,@MileyCyrus @CashApp $SantySmiler Apple @CashA...,0,2021-03-31 19:14:26,#INVESTINYOU
...,...,...,...,...,...,...
2495,1377325677901398019,eclt11,@MileyCyrus @CashApp my cash app tag is $guap7...,0,2021-03-31 18:23:25,#INVESTINYOU
2496,1377325667830882305,oohitscathy,@MileyCyrus @CashApp #INVESTINYOU #partner #15...,0,2021-03-31 18:23:23,#INVESTINYOU
2497,1377325658666381315,1stcomesfamily,RT @pika_saur: @MileyCyrus @CashApp Thank you ...,32,2021-03-31 18:23:21,#INVESTINYOU
2498,1377325645248757762,sydney_easton_,RT @MileyCyrus: Nothing is more important than...,4065,2021-03-31 18:23:17,#INVESTINYOU


# Ngrams

This step is about building a NN model, to understand Ngrams